# Классификаиция клиентов телеком компании

## Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.\
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.\
Известно:\
`сalls` — количество звонков,\
`minutes` — суммарная длительность звонков в минутах,\
`messages` — количество sms-сообщений,\
`mb_used` — израсходованный интернет-трафик в Мб,\
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## План проекта
1. [Импорт данных и изучение датасета](#import) 
2. [Разделение исходных данных на обучающую, валидационную и тестовую выборки.](#split)
3. [Исследование качества разных моделей, с изменением гиперпараметров](#research)\
    3.1 [Дерево решений](#DecisionTreeClassifier)\
    3.2 [Cлучайный лес](#RandomForestClassifier)\
    3.3 [Логистическая регрессия](#LogisticRegression)
4. [Проверка качества модели на тестовой выборке.](#test)
5. [Дополнительное задание: проверка модели на вменяемость.](#sanity)
6. [Выводы](#concl)

### 1. Импорт данных и изучение датасета <a id='import'></a> 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
import numpy as np
import pandas as pd

In [2]:
try:
    df = pd.read_csv('C:/Users/79153/Desktop/ya_projects/sprint_9/users_behavior.csv')
except:
    df = pd.read_csv('datasets/users_behavior.csv')

Просмотрим первые 10 строк датасета, информацию методом info() и describe()

In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Вывод по первичному изучению датасета
Данные действительно обработанные, можно было бы заменить тип данных на *float32* в соответствующих столбцах, но ввиду малого объема данных такая работа не принесет много пользы, кроме того в условиях задачи написано, что данные обработаны и никаких действий по предобработке не требуют.

### 2. Разделение исходных данных на обучающую, валидационную и тестовую выборки. <a id='split'></a>

In [6]:
main_df, df_test = train_test_split(df, test_size=0.20, random_state=12345)               # отбор тестовой выборки
df_train, df_valid = train_test_split(main_df, test_size=0.20/0.8, random_state=12345)    # отбор валидационной выборки

В данном случае у нас нет скрытой выборки и было применено разделение в соотношении 3:1:1.

### 3. Исследование качества разных моделей, с изменением гиперпараметров  <a id='research'></a>

Объявим переменные с признаками для обучения, валидации и теста 

In [7]:
# переменные с признаком для обучения и целью 
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
# то же для валидации
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
# то же для теста
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [8]:
print('''Размеры выборки для обучения модели: features {} target {}
Размеры выборки для валидации модели: features {} target {}
Размеры выборки для теста модели: features {} target {}'''.format(features_train.shape, len(target_train),
                                               features_valid.shape, len(target_valid),
                                               features_test.shape, len(target_test)))

Размеры выборки для обучения модели: features (1928, 4) target 1928
Размеры выборки для валидации модели: features (643, 4) target 643
Размеры выборки для теста модели: features (643, 4) target 643


#### 3.1 Дерево решений <a id='DecisionTreeClassifier'></a>

In [9]:
best_params = []
best_accuracy = 0.5
# проход по гиперпараметрам
for max_depth in range(2,10):
    for min_samples_split in range(2,10):
        for min_samples_leaf in range(1,10):
            for criterion in ['gini', 'entropy']:
                # инициация модели 
                model = DecisionTreeClassifier(random_state=123,
                                               max_depth=max_depth,
                                               min_samples_split=min_samples_split,
                                               min_samples_leaf=min_samples_leaf,
                                               criterion=criterion)
                # обучение модели 
                model.fit(features_train,target_train)
                # поиск предсказаний модели на валидационной выбоке
                predictions = model.predict(features_valid)
                # вычисление точности модели методом accuracy_score
                accuracy = accuracy_score(target_valid, predictions)
                # сохранение лучшего результата
                if best_accuracy < accuracy:
                    best_params = [max_depth, min_samples_split, min_samples_leaf, criterion]
                    best_accuracy = accuracy

In [10]:
# вывод модели.
print('''Наилучший показатель accuracy: {:.4f}
Параметры модели:
   max_depth = {}
   min_samples_split = {}
   min_samples_leaf = {}
   criterion = {} '''.format(best_accuracy,
                             best_params[0],
                             best_params[1],
                             best_params[2],
                             best_params[3],
                            ))

Наилучший показатель accuracy: 0.7963
Параметры модели:
   max_depth = 8
   min_samples_split = 2
   min_samples_leaf = 5
   criterion = entropy 


#### 3.2 Cлучайный лес <a id='RandomForestClassifier'></a>

In [11]:
from sklearn.ensemble import RandomForestClassifier
best_params = []
best_accuracy = 0
# проход по гиперпараметрам
for n_estimators in (1,10):
    for max_depth in range(2,10):
        for min_samples_split in range(2,10):
            for min_samples_leaf in range(1,8):
                for criterion in ['gini', 'entropy']:
                    # инициация модели 
                    model = RandomForestClassifier(random_state=1234,
                                                   n_estimators=n_estimators,
                                                   max_depth=max_depth,
                                                   min_samples_split=min_samples_split,
                                                   min_samples_leaf=min_samples_leaf,
                                                   criterion=criterion)
                    # обучение модели 
                    model.fit(features_train,target_train)
                    # поиск предсказаний модели на валидационной выбоке
                    predictions = model.predict(features_valid)
                    # вычисление точности модели методом accuracy_score
                    accuracy = accuracy_score(target_valid, predictions)
                    # сохранение лучшего результата
                    if accuracy > best_accuracy:
                        best_params_2 = [n_estimators, max_depth, min_samples_split, min_samples_leaf, criterion]
                        best_accuracy = accuracy

In [12]:
# вывод наилучшей точности и параметров модели.
print('''Наилучший показатель accuracy: {:.4f}
Параметры модели:
   n_estimators = {}
   max_depth = {}
   min_samples_split = {}
   min_samples_leaf = {}
   criterion = {} '''.format(best_accuracy,
                             best_params_2[0],
                             best_params_2[1],
                             best_params_2[2],
                             best_params_2[3],
                             best_params_2[4]))

Наилучший показатель accuracy: 0.8009
Параметры модели:
   n_estimators = 10
   max_depth = 9
   min_samples_split = 7
   min_samples_leaf = 1
   criterion = entropy 


#### 3.3 Логистическая регрессия <a id='LogisticRegression'></a>

In [13]:
# в параметрах best_params и best_accuracy будем хранить наилучшие параметры для модели и наивысшую точность соответственно
best_params = []
best_accuracy = 0

cs = np.linspace(0.01, 10, 1000, endpoint=True)
# проход по выбранным гиперпараметрам
for C in cs:
    # инициация модели 
    model = LogisticRegression(random_state=12345,
                                      C=C,
                                      max_iter=1000)
    # обучение модели 
    model.fit(features_train,target_train)
    # поиск предсказаний модели на валидационной выбоке
    predictions = model.predict(features_valid)
    # вычисление точности модели
    accuracy = accuracy_score(target_valid, predictions)
    # сохранение лучшего результата
    if accuracy > best_accuracy:
        best_params_3 = [C]
        best_accuracy = accuracy

In [15]:
# вывод наилучшей точности и параметров модели.
print('''Наилучший показатель accuracy: {:.4f}
Параметеры модели:
   C = {} '''.format(best_accuracy,
                       best_params_3[0]))

Наилучший показатель accuracy: 0.7278
Параметеры модели:
   C = 2.94 


#### Промежуточный вывод
Наилучший результат показала модель *RandomForestClassifier* c accuracy = 0.8009. 

### 4. Проверка качества модели на тестовой выборке. <a id='test'></a>

In [16]:
model = RandomForestClassifier(random_state=1234,
                               n_estimators=best_params_2[0],
                               max_depth=best_params_2[1],
                               min_samples_split=best_params_2[2],
                               min_samples_leaf=best_params_2[3],
                               criterion=best_params_2[4])
model.fit(features_train,target_train)
predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print('''Точность модели на тестовой выборке: {:.4f}'''.format(accuracy))

Точность модели на тестовой выборке: 0.7932


Проверку на качество на тестовой выборке модель проходит.

### 5. Дополнительное задание: проверка модели на вменяемость. <a id='sanity'></a>
Для проверки вменяемости модели попробуем использовать тестовую выборку с с заменой всех значений тарифов на 1 и 0, а так же проверим точность с помощью DummyClassifier

In [17]:
X = features_test
y = target_test
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy='most_frequent')
print('''Точность dummy на тестовой выборке: {:.4f}'''.format(dummy_clf.score(X, y)))

Точность dummy на тестовой выборке: 0.6952


In [18]:
predictions = model.predict(features_test)
false_target = []
for i in range(len(predictions)):
    false_target.append(0)
accuracy_2 = accuracy_score(false_target, predictions)
print('''Точность модели на тестовой выборке: {:.4f}'''.format(accuracy_2))

Точность модели на тестовой выборке: 0.7963


In [19]:
predictions = model.predict(features_test)
false_target = []
for i in range(len(predictions)):
    false_target.append(1)
accuracy_2 = accuracy_score(false_target, predictions)
print('''Точность модели на тестовой выборке: {:.4f}'''.format(accuracy_2))

Точность модели на тестовой выборке: 0.2037


Точность модели выше точности dummy, так же можно отметить некоторое повышение эффективности модели при проверке на фальш целью. Вероятно в затруднительной ситуации модель возвращает наиболее частый результат.

### 6. Выводы <a id='concl'></a>
В данном исследовании были рассмотрены модели машинного обучения *DecisionTreeClassifier*, *RandomForestClassifier* и *LogisticRegression*.

Наибольшую точность показала модель *RandomForestClassifier*, однако проверку на вменяемость она проходит так как точность выше точности *DummyClassifyer*. 

Таким образом можно сделать вывод, что при выборе модели обучения следует проводить анализ различных моделей с учетом различных параметров, а так же скорости реализации. Модель *DecisionTreeClassifier* получила результат не намного менее точный чем *RandomForestClassifier*, но при этом скорость работы данной модели намного быстрее.